# Pandas

In [1]:
from flypipe import node
from flypipe.schema import Schema, Column
from flypipe.schema.types import String
import pandas as pd

# Create the Graph
@node(
    type="pandas",
    output=Schema(
     Column("_fruit", String(), "_fruit"),
    )
)
def table():
    return pd.DataFrame(data={'_fruit': ['ORANGE', 'WATERMELON', 'LEMON']})



@node(
    type="pandas",
    dependencies=[
        table.select("_fruit").alias("df")
    ],
    output=Schema(
     Column("fruit", String(), "fruit description"),
    )
)
def clean(df):
    df = df.rename(columns={'_fruit': 'fruit'})
    df['fruit'] = df['fruit'].str.lower()
    return df

@node(
    type="pandas",
    dependencies=[
       clean.select("fruit").alias("df")
    ],
    output=Schema(
        Column("fruit", String(), "fruit description"),
        Column("color", String(), "color of the fruit"),
    )
)
def color(df):
    
    replacements = {
        "orange": "orange",
        "watermelon": "red",
        "lemon": "yellow",
    }
    
    df['color'] = df['fruit']
    df = df.replace({"color": replacements})
    return df

@node(
    type="pandas",
    dependencies=[
       clean.select("fruit").alias("df")
    ],
    output=Schema(
        Column("fruit", String(), "fruit description"),
        Column("category", String(), "category of the fruit"),
    )
)
def category(df):
    
    replacements = {
        "orange": "citric",
        "watermelon": "sweet",
        "lemon": "citric",
    }
    
    df['category'] = df['fruit']
    df = df.replace({"category": replacements})
    return df


@node(
    type="pandas",
    dependencies=[
       color.select("fruit", "color"),
       category.select("fruit", "category")  
    ],
    output=Schema(
        Column("fruit", String(), "fruit description"),
        Column("color", String(), "color of the fruit"),
        Column("category", String(), "category of the fruit"),
    )
)
def fruits(color, category):
    return color.merge(category, on="fruit", how="left")


# Run 

df = fruits.run()
print(df)


        fruit   color category
0      orange  orange   citric
1  watermelon     red    sweet
2       lemon  yellow   citric
